### Day 17: Chronospatial Computer

Link: https://adventofcode.com/2024/day/17#part2

Note: This analysis might contain elements specific to my input, as I do not have access to other inputs to determine how much more I should generalize it.

To solve part two, we need some insights from the input:

1. The program only potentially jumps at the last opcode. When it does so, it goes back to the first opcode. Let's call each passing through the program an iteration.
2. Initial values in registers B and C on each iteration don't matter, as they are derived from the value in register A.
3. The value from register A is floor-divided by 2 to the power of the combo operand on each iteration. I assume all inputs would have this operand between 0 and 3, inclusive, so they are treated as literal. In my case, it's operand 3, so A is divided by 8 every time.
4. The final value of register A must be zero for the program to halt.

Given these conditions, we can work the program backward to calculate possible candidates for register A on each iteration. For example, on the first backward iteration, register A's value must end with zero. Since it's floor-divided by 8, that means it has to start with a number between 0 and 7, inclusive. That range is given by `[A * 8, (A + 1) * 8 - 1]`. We then test all these options and add the ones that give us the expected output to the list of candidates. Then, on the next iteration, we look within a new range of options, `[A * 8, (A + 1) * 8 - 1]`, where A is each possible candidate found in the previous iteration. We keep processing until we find the final candidates for register A's value. Given the list of candidates should be sorted, we simply provide the smallest one, the first, as the answer. We can validate our answer running it against the solution from part one and checking if it generates a copy of the program itself.

In [1]:
# Please ensure there is an `input.txt` file in this folder containing your input.
with open("input.txt", "r") as file:
    lines = file.readlines()

In [ ]:
program = [int(value) for value in lines[4].split(":")[-1].strip().split(",")]


def get_literal_operand_value(operand: int) -> int:
    return operand


def get_combo_operand_value(operand: int) -> int:
    if 0 <= operand <= 3:
        return operand

    if operand == 4:
        return register_a_value

    if operand == 5:
        return register_b_value

    if operand == 6:
        return register_c_value

    raise Exception("Invalid operand")


def adv(operand: int) -> str:
    global register_a_value
    global instruction_pointer
    operand_value = get_combo_operand_value(operand)
    register_a_value = register_a_value // pow(2, operand_value)
    instruction_pointer += 2
    return ""


def bxl(operand: int) -> str:
    global register_b_value
    global instruction_pointer
    operand_value = get_literal_operand_value(operand)
    register_b_value = register_b_value ^ operand_value
    instruction_pointer += 2
    return ""


def bst(operand: int) -> str:
    global register_b_value
    global instruction_pointer
    operand_value = get_combo_operand_value(operand)
    register_b_value = operand_value % 8
    instruction_pointer += 2
    return ""


def jnz(operand: int) -> str:
    global instruction_pointer

    if register_a_value == 0:
        instruction_pointer += 2
        return ""

    operand_value = get_literal_operand_value(operand)
    instruction_pointer = operand_value
    return ""


def bxc(operand: int) -> str:
    global register_b_value
    global instruction_pointer
    register_b_value = register_b_value ^ register_c_value
    instruction_pointer += 2
    return ""


def out(operand: int) -> str:
    global instruction_pointer
    operand_value = get_combo_operand_value(operand) % 8
    instruction_pointer += 2
    return str(operand_value)


def bdv(operand: int) -> str:
    global register_b_value
    global instruction_pointer
    operand_value = get_combo_operand_value(operand)
    register_b_value = register_a_value // pow(2, operand_value)
    instruction_pointer += 2
    return ""


def cdv(operand: int) -> str:
    global register_c_value
    global instruction_pointer
    operand_value = get_combo_operand_value(operand)
    register_c_value = register_a_value // pow(2, operand_value)
    instruction_pointer += 2
    return ""


instructions = {
    0: adv,
    1: bxl,
    2: bst,
    3: jnz,
    4: bxc,
    5: out,
    6: bdv,
    7: cdv,
}
register_a_value = 0
register_b_value = 0
register_c_value = 0
potential_a_values = [register_a_value]


for expected_output in reversed(program):
    new_potential_a_values: list[int] = []

    for potential_a_value in potential_a_values:
        for new_potential_a_value in range(potential_a_value * 8, (potential_a_value + 1) * 8):
            register_a_value = new_potential_a_value
            instruction_pointer = 0
            output = -1

            while instruction_pointer < len(program) - 2:
                opcode = program[instruction_pointer]
                operand = program[instruction_pointer + 1]
                instruction = instructions[opcode]
                result = instruction(operand)

                if result:
                    output = int(result)

            if output == expected_output:
                new_potential_a_values.append(new_potential_a_value)

    potential_a_values = new_potential_a_values


print(potential_a_values[0])